### Demo: predicción de símbolos matemáticos escritos a mano
---
_Autor: Beñat Berasategui Miguéliz_.

**Instrucciones:** Para ver un ejemplo, pulsar el botón *ejemplo* y después *predecir*. Pulsar *borrar*. Para introducir un símbolo, se debe desplazar el ratón hasta la sección gris. Para dibujar hacer click izquierdo y, sin soltar, desplazar el ratón. Pulsar en *predecir*. El funcionamiento óptimo se obtiene dibujando los símbolos de manera que ocupen el mayor espacio posible en la zona gris, y con el tamaño de brocha por defecto. Ver la parte inferior de la página para más información.

In [18]:
import sklearn
from tensorflow import keras
import cv2
import numpy as np
import ipywidgets as widgets
import pickle 
import itertools

In [14]:
# Cargar diccionarios necesarios
with open('index2symbol_id_test.pkl', 'rb') as f:
    index2symbol_id_test = pickle.load(f) 
with open('symbolid2latex.pkl', 'rb') as f:
    symbolid2latex = pickle.load(f)
    
# Modelo con base de datos original:
# path_original = '/home/jovyan/work/models/04_CNN/00_MNIST_model/Test_performance/models/best_model.h5'
model_original_data = keras.models.load_model('modelo_original.h5')

# Modelo con base de datos aumentada:
# path_aug = '/home/jovyan/work/models/05_CNN_aug/aug_1000/models_aug/best_model.h5'
model_aug_data = keras.models.load_model('modelo_augmented.h5')

In [19]:
# Diccionario de símbolos equivalentes (ver hasy_tools_updated.py)

equiv_dict={
    0 : (r'\mid', r'|'),
    1 : (r'\triangle', r'\Delta', r'\vartriangle'),
    2 : (r'\checked', r'\checkmark'),
    3 : (r'\shortrightarrow', r'\rightarrow', r'\longrightarrow'),
    4 : (r'\Longrightarrow', r'\Rightarrow'),
    5 : (r'\backslash', r'\setminus'),
    6 : (r'\O', r'\o', r'\emptyset', r'\diameter', r'\varnothing'),
    7 : (r'\with', r'\&'),
    8 : (r'\triangledown', r'\nabla'),
    9 : (r'\longmapsto', r'\mapsto'),
    10 : (r'\dotsc', r'\dots'),
    11 : (r'\fullmoon', r'\circ', r'o', '\degree'),
    12 : (r'\varpropto', r'\propto', r'\alpha'),
    13 : (r'\mathsection', r'\S'),
    14 : (r'\vDash', r'\models'),
    15 : (r'c', r'C', r'\subset', r'\mathcal{C}'),
    16 : (r'v', r'V', r'\vee'),
    17 : (r'x', r'X', r'\times'),
    18 : (r'\mathbb{Z}', r'\mathds{Z}'),
    19 : (r'T', r'\top'),
    20 : (r's', r'S', r'\mathcal{S}'),
    21 : (r'z', r'Z', r'\mathcal{Z}'),
    22 : (r'\mathbb{R}', r'\mathds{R}'),
    23 : (r'\mathbb{Q}', r'\mathds{Q}'),
    24 : (r'\mathbb{N}', r'\mathds{N}'),
    25 : (r'\oiint', r'\varoiint'),
    26 : (r'\lhd', r'\triangleleft'),
    27 : (r'\sum', r'\Sigma'),
    28 : (r'\prod', r'\Pi', r'\sqcap'),
    29 : (r'\mathcal{B}', r'B'),
    30 : (r'\mathcal{D}', r'D', r'\mathscr{D}'),
    31 : (r'\mathcal{H}', r'H'),
    32 : (r'\mathcal{M}', r'M'),
    33 : (r'\mathcal{N}', r'N', r'\mathscr{N}'),
    34 : (r'\mathcal{O}', r'O', r'0'),
    35 : (r'\mathcal{P}', r'P'),
    36 : (r'\mathcal{R}', r'R', r'\mathscr{R}'),
    37 : (r'\coprod', r'\amalg', r'\sqcup'),
    38 : (r'\bot', r'\perp'),
    39 : (r'\|', r'\parallel'),
    40 : (r'\ohm', r'\Omega'),
    41 : (r'\#', r'\sharp'),
    42 : (r'\mathcal{A}', r'\mathscr{A}'),
    43 : (r'\epsilon', r'\varepsilon', r'\in', r'\mathcal{E}'),
    44 : (r'\Lambda', r'\wedge'),
    45 : (r'\Leftrightarrow', r'\Longleftrightarrow'),
    46 : (r'\mathds{1}', r'\mathbb{1}'),
    47 : (r'\mathscr{L}', r'\mathcal{L}'),
    48 : (r'\rho', r'\varrho'),
    49 : (r'\odot', r'\astrosun'),
    50 : (r'\cdot', r'\bullet'),
    51 : (r'\chi', r'\mathcal{X}'),
    52 : (r'\beta', r'\ss'),
    53 : (r'\male', r'\mars'),
    54 : (r'\female', r'\venus'),
    55 : (r'\bowtie', r'\Bowtie'),
    56 : (r'\mathcal{T}', r'\tau'),
    57 : (r'\diamond', r'\diamondsuit', r'\lozenge'),
}

# List all the symbols in the equivalence dictionary:
equiv_list = list(itertools.chain.from_iterable(equiv_dict.values()))

# Invert the dictionary to find equivalent symbols easier:
inverted_dict = {value: key for key, values in equiv_dict.items() for value in values}

In [3]:
def predict(drawing_widget, model, printlog=None, n=5):
    image_output = drawing_widget.get_image_data()
    img = ((image_output[:,:,-1]/255.0+1)%2).astype(dtype='uint8')
    res = cv2.resize(img, dsize=(32, 32), interpolation=cv2.INTER_AREA)

    X_mine = np.zeros((1, 32, 32))
    X_mine[0] = res
    X_mine = X_mine[..., np.newaxis]

    y_probs = model.predict(X_mine, verbose=printlog) # Probabilities
    y_pred = np.argmax(y_probs, axis=-1) # Predictions
        
    y_preds = np.argsort(y_probs, axis=1)[:,-n:]
    probs = 100*np.sort(y_probs, axis=1)[:,-n:]
    command_list = []
    prob_list = []
    
    for i in range(len(y_probs)):
        latex = [symbolid2latex[index2symbol_id_test[k]] for k in y_preds[i][::-1]]
        for j in range(0,n):
            command_list.append(latex[j])
            prob_list.append(probs[i][-1-j])

    return command_list, prob_list

In [27]:
def actualizar_prediccion():
    global command_dict, prob_dict
        
    original_model_out = predict(drawing_widget, model_original_data)
    augmented_model_out = predict(drawing_widget, model_aug_data)
    command_dict = {'original':original_model_out[0], 'augmented':augmented_model_out[0]}
    prob_dict = {'original':original_model_out[1], 'augmented':augmented_model_out[1]}
    
    with output:
        output.clear_output(wait=True)
        display(refresh_layout())
        
    with drawing_widget.output_dropdown:
        drawing_widget.output_dropdown.clear_output(wait=True)
        display(refresh_dropdown())

In [40]:
def fig_name_from_command(command):
    if command in ('|','\|','/'):
        if command == '|':
            fig_name = '/especial/textpipe'
        elif command == '\|':
            fig_name = '/especial/|'
        else:
            fig_name = '/especial/textslash'
    elif command in ('\o','\O'):
        if command == '\o':
            fig_name = 'o_'
        else:
            fig_name = 'O_'
    elif command[0] == '\\':
        fig_name = command[1:]
    else:
        fig_name = command
    return fig_name

def load_img_from_command(command):
    fig_name = fig_name_from_command(command)
    try:
        fig = widgets.Image(value=open(f"png/{fig_name}.png", "rb").read(),format='png')
    except:
        # Empty image if there is any error.
        fig = widgets.Image(value=open("png/empty.png", "rb").read(),format='png')
    finally:
        return fig

In [42]:
def dropdown_eventhandler(change):
    if change['type'] == 'change' and change['name'] == 'value':
        fig = load_img_from_command(change['new'])
        fig.layout.margin = '10px 0 0 80px'
        fig.layout.height='100px'
        fig.layout.object_fit = 'contain'
        with drawing_widget.output_img:
            drawing_widget.output_img.clear_output(wait=True)
            display(fig)

In [48]:
def refresh_dropdown():
    command = command_dict['augmented'][0]
    
    if command not in equiv_list:
        fig = load_img_from_command(command)
        dropdown = widgets.Label(value=command) #widgets.Dropdown(options=[(command, 1)],value=1)
        dropdown.layout.margin = '0 0 0 20px'
        
    else:
        key = inverted_dict[command]
        equiv_symbs = equiv_dict[key]
        dropdown = widgets.Dropdown(
                        options=equiv_symbs,
                        value=equiv_symbs[0],
                        )
        fig = load_img_from_command(equiv_symbs[0])
        
    dropdown.observe(dropdown_eventhandler)
    
    with drawing_widget.output_img:
        fig.layout.margin = '10px 0 0 80px'
        fig.layout.height = '100px'
        fig.layout.object_fit = 'contain'
        
        drawing_widget.output_img.clear_output(wait=True)
        display(fig)
        
    return dropdown

In [49]:
def refresh_layout():
    fig_dict = dict() 
    text_dict = dict()
    text_prob_dict = dict()
    widg_dict = dict()

    titles = {'original':"Predicciones del sistema original", 'augmented':"Predicciones del sistema mejorado"}

    for model in ('original', 'augmented'):
        fig_dict[model] = []
        text_dict[model] = []
        text_prob_dict[model] = []
        widg_dict[model] = []

        for command in command_dict[model]:
            text_dict[model].append(widgets.Label(value = f"{command}",style = {'font_size':'15pt', 'font_family':'monospace'}))
            # Process fig_name to open the correct png.
            #command = command.strip()
            fig_name = fig_name_from_command(command)
            try:
                fig = widgets.Image(value=open(f"png/{fig_name}.png", "rb").read(),format='png',width=20,height=20,)
            except:
                # Empty image if there is any error.
                fig = widgets.Image(value=open("png/empty.png", "rb").read(),format='png',width=20,height=20,)
            finally:
                fig_dict[model].append(fig)

        for prob in prob_dict[model]:
            if type(prob) != str:
                text_prob_dict[model].append(widgets.Label(value = f"{prob:.2f}%",style = {'font_size':'15pt', 'font_family':'monospace'}))
            else:
                text_prob_dict[model].append(widgets.Label(value = f"     ",style = {'font_size':'13pt', 'font_family':'monospace'}))

        widg_dict[model].append(widgets.Label(value = titles[model], style = {'font_size':'13pt', 'font_weight':'bold'}))
    
        for i in range(5):
            widg_dict[model].append(widgets.AppLayout(
                      left_sidebar=fig_dict[model][i],
                      center=text_prob_dict[model][i],
                      right_sidebar=text_dict[model][i],
                      pane_widths=[1, 2, 4],
                      align_items='center',
                      justify_items='flex-start',
                      height="40px", width="80%",
                      grid_gap="10px"))
            
        
    #return widgets.HBox([widgets.VBox(widg_dict['original']), widgets.VBox(widg_dict['augmented'])])
    sistema_original = widgets.VBox(widg_dict['original'])
    sistema_aumentado = widgets.VBox(widg_dict['augmented'])
    sistema_original.layout.flex = '1'
    sistema_aumentado.layout.flex = '1'
    
    display_predictions = widgets.HBox([sistema_original, sistema_aumentado])
    #display_predictions.layout.width = '2000px' 
    
    return display_predictions

In [12]:
from drawing_widget_modified import DrawingWidget
drawing_widget = DrawingWidget(width=200, height=200, external_function=actualizar_prediccion)

In [13]:
drawing_widget.show()

In [8]:
# from IPython.display import display

# Initialize empty
command_dict = {'original':[' ']*5, 'augmented':[' ']*5}
prob_dict = {'original':['']*5, 'augmented':[' ']*5}

output = widgets.Output()
output.append_display_data(refresh_layout())
display(output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(children=(VBox(children=(Label(va…

---
**Más información**

Hay ciertos símbolos que resultan prácticamente indistinguibles cuando se escriben a mano. En este proyecto se han definido varios grupos de símbolos equivalentes de este tipo. Cuando el sistema predice un símbolo que pertenece a uno de estos grupos, aparece un menú desplegable que permite elegir entre los elementos del grupo y muestra la imagen correspondiente.

El *sistema original* se ha entrenado utilizando la base de datos [HASY](https://arxiv.org/pdf/1701.08380), sin arreglar el desequilibrio entre las clases. Hay clases que contienen más de 1000 muestras y otras que solo tienen 50. Esto se refleja en el funcionamiento del modelo obtenido, ya que hay clases que se predicen con mayor probabilidad. El _sistema mejorado_ se ha entrenado con una base de datos en la que se ha utilizado la técnica llamada "aumento de datos". De esta manera, se ha conseguido que todas las clases tengan el mismo número de muestras: 1000. Se espera que el funcionamiento general de este segundo sistema sea mejor que el del primero. Por eso, es el que se utiliza para dar la predicción principal en la parte superior derecha.

Esta demo forma parte de mi Trabajo de Fin de Grado de Ingeniería Electrónica, *Estudio, desarrollo y evaluación de técnicas de aprendizaje automático
para el reconocimiento de símbolos matemáticos escritos a mano*.

La aplicación está inspirada en [Detextify](https://detexify.kirelabs.org/).